In [42]:
from astropy import coordinates,units,constants
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import ascii

In [47]:
## load all satellite galaxies
data = ascii.read("./Drlica-Wagner2020.txt")

## let's calculate the angular size of these galaxies
distance=np.array(data['D'])## distance of these galaxies in kpc


### Crab pulsar lets say 1kJy for a reasonable example?
crabgp=1000*units.Jy 


crab=(7175*units.lightyear).to(units.kpc)
crab_pf=14*units.mJy.to(units.Jy)

vela_pf=1100*units.mJy.to(units.Jy)
vela=0.28*units.kpc

In [53]:
mockcrab=crabgp*crab**2/(distance*units.kpc)**2
print('Crab GP at 10kpc:',crabgp*crab**2/(10*units.kpc)**2)

Crab GP at 10kpc: 48.394060201105525 Jy


In [49]:
data.add_column(col=mockcrab,name='Example Crab GP luminosity')

In [50]:
data

Name,Survey,Class,RAdeg,DEdeg,m-M,ah,l_e,e,D,r1/2,VMag,Ref,Example Crab GP luminosity
,,,deg,deg,mag,arcmin,,,kpc,pc,mag,,Jy
str17,str8,int64,float64,float64,float64,float64,str1,float64,int64,int64,float64,str6,float64
Antlia II,--,4,143.8868,-36.7673,20.6,76.2,--,0.38,132,2301,-9.03,1,0.27774368802287375
Aquarius II,PS1,4,338.4813,-9.3274,20.2,5.1,--,0.39,108,125,-4.4,2,0.4149010648242929
Bootes I,PS1,4,210.02,14.5135,19.1,9.97,--,0.3,66,160,-6.02,3,1.110974752091495
Bootes II,PS1,4,209.5141,12.8553,18.1,3.17,--,0.25,42,33,-2.94,3,2.743427449042263
Bootes III,PS1,4,209.3,26.8,18.4,30.0,--,0.5,47,289,-5.75,4,2.1907677773248313
Bootes IV,PS1,3,233.689,43.726,21.6,7.6,--,0.64,209,277,-4.53,5,0.11078972597034299
Canes Venatici I,PS1,4,202.0091,33.5521,21.7,7.12,--,0.44,218,338,-8.8,3,0.10183078066052
Canes Venatici II,PS1,4,194.2927,34.3226,21.0,1.52,--,0.4,160,55,-5.17,3,0.18903929766056846
